<a href="https://colab.research.google.com/github/kurianbenoy/Cactus-Challenge--Kaggle/blob/master/Learning_throughAerial_CactusChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import CSVLogger, SaveModelCallback
from pathlib import Path
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import torch

In [0]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/sample_submission.csv")

In [0]:
data_folder = Path("../input/")

In [0]:
test_img = ImageList.from_df(test_df, path=data_folder/'test', folder='test')
trfm = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)
train_img = (ImageList.from_df(train_df, path=data_folder/'train', folder='train')
        .split_by_rand_pct(0.01)
        .label_from_df()
        .add_test(test_img)
        .transform(trfm, size=128)
        .databunch(path='.', bs=64, device= torch.device('cuda:0'))
        .normalize(imagenet_stats)
       )

In [0]:
learn = cnn_learner(train_img, models.densenet161, metrics=[error_rate, accuracy])

In [0]:
learn.lr_find()
learn.recorder.plot()


In [0]:
lr = 3e-02
learn.fit_one_cycle(5, slice(lr))

In [0]:
learn_n = cnn_learner(data, models.resnet18, metrics=accuracy)